### Importing all the required packages

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

### Importing data

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


### Lists for holding data

In [4]:

postalCodeList = []
boroughList = []
neighborhoodList = []

### Extracting data using beautiful soup

In [5]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell


### Creating dataframe using our data lists

In [6]:
toronto_data = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})


In [7]:
toronto_data.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Filtering Borough that has "Not assigned" as value

In [8]:
toronto_data_filter = toronto_data[toronto_data.Borough != 'Not assigned'].reset_index(drop = True)

In [9]:
toronto_data_filter.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [10]:
### Grou

In [11]:
toronto_grouped_data = toronto_data_filter.groupby(["PostalCode","Borough"],as_index = False).agg(lambda x: ",".join(x))

In [12]:
toronto_grouped_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
toronto_grouped_data.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

### Only one row has the Neighborhood value as "Not assigned"

In [14]:
toronto_grouped_data[toronto_grouped_data.Neighborhood == "Not assigned"]

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


### Replace the neighborhood value with Borough itself

In [15]:
for index, row in toronto_grouped_data.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_grouped_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Check for Neighborhood having "Not assigned"

In [16]:
toronto_grouped_data[toronto_grouped_data.Neighborhood == "Not assigned"]

,PostalCode,Borough,Neighborhood


In [17]:
toronto_grouped_data.shape

(103, 3)

### Importing coordinates

In [18]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [19]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Joining both the dataframes

In [22]:
toronto_merged = toronto_grouped_data.merge(coordinates, on="PostalCode", how="left")
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
!pip install geopy

     |████████████████████████████████| 102kB 165kB/s ta 0:00:01


In [26]:
import json
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### Getting coordinates

In [28]:
address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating Toronto Map

In [31]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Get Borough with Toronto

In [32]:
toronto_merged.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

In [33]:
filtered_boroughs = [borough for borough in list(toronto_merged.Borough.unique()) if "toronto" in borough.lower()]

In [34]:
filtered_boroughs

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

### Toronto dataframe

In [36]:
toronto_dataframe = toronto_merged[toronto_merged.Borough.isin(filtered_boroughs)].reset_index(drop = True)

In [37]:
toronto_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Plotting map again

In [41]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_dataframe['Latitude'], toronto_dataframe['Longitude'], toronto_dataframe['Borough'], toronto_dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [42]:
# define Foursquare Credentials and Version
CLIENT_ID = 'RE3GEXCG5HCQKT51QWR1VOXQQUB0HS5LTFNI1A4QJQHL5VOW' # your Foursquare ID
CLIENT_SECRET = 'WKFKMAVRI4NPGINIHDITKBXCKDP3OBVOTXOEVXDECNA50PRM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RE3GEXCG5HCQKT51QWR1VOXQQUB0HS5LTFNI1A4QJQHL5VOW
CLIENT_SECRET:WKFKMAVRI4NPGINIHDITKBXCKDP3OBVOTXOEVXDECNA50PRM


In [43]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_dataframe['Latitude'], toronto_dataframe['Longitude'], toronto_dataframe['PostalCode'], toronto_dataframe['Borough'], toronto_dataframe['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [44]:
venues_df = pd.DataFrame(venues)
# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1726, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [45]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West,Riverdale",42,42,42,42,42,42
M4L,East Toronto,"The Beaches West,India Bazaar",21,21,21,21,21,21
M4M,East Toronto,Studio District,42,42,42,42,42,42
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,34,34,34,34,34,34
M4T,Central Toronto,"Moore Park,Summerhill East",2,2,2,2,2,2


In [46]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


In [48]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1726, 243)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West,Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(39, 243)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.25000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.023810,...,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.02381
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,...,0.00000,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.02381
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000


In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Health Food Store,Trail,Pub,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",Park,Sandwich Place,Movie Theater,Brewery,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Brewery,Yoga Studio,Sandwich Place,Fish Market
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
5,M4P,Central Toronto,Davisville North,Gym,Breakfast Spot,Park,Food & Drink Shop,Sandwich Place,Clothing Store,Hotel,Dumpling Restaurant,Dog Run,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Cosmetics Shop,Chinese Restaurant,Dessert Shop,Rental Car Location,Restaurant,Diner
7,M4S,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park,Summerhill East",Tennis Court,Summer Camp,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,Pizza Place,Liquor Store,Sports Bar,Supermarket,Sushi Restaurant,Restaurant,Fried Chicken Joint,Bagel Shop


### Clustering

In [51]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 1, 0], dtype=int32)

In [53]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_dataframe.copy()
# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Movie Theater,Brewery,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Brewery,Yoga Studio,Sandwich Place,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [55]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Health Food Store,Trail,Pub,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Movie Theater,Brewery,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Brewery,Yoga Studio,Sandwich Place,Fish Market
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Breakfast Spot,Park,Food & Drink Shop,Sandwich Place,Clothing Store,Hotel,Dumpling Restaurant,Dog Run,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Cosmetics Shop,Chinese Restaurant,Dessert Shop,Rental Car Location,Restaurant,Diner
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Seafood Restaurant
19,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Scenic Lookout,Restaurant,Brewery,Fried Chicken Joint,Pizza Place
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop,Gym / Fitness Center
11,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Coffee Shop,Bakery,Restaurant,Pizza Place,Pub,Café,Italian Restaurant,Convenience Store,Gastropub,Pharmacy


In [56]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Neighborhood,Health Food Store,Trail,Pub,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant,Falafel Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Caribbean Restaurant
2,East Toronto,0,Park,Sandwich Place,Movie Theater,Brewery,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Brewery,Yoga Studio,Sandwich Place,Fish Market
5,Central Toronto,0,Gym,Breakfast Spot,Park,Food & Drink Shop,Sandwich Place,Clothing Store,Hotel,Dumpling Restaurant,Dog Run,Doner Restaurant
6,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Cosmetics Shop,Chinese Restaurant,Dessert Shop,Rental Car Location,Restaurant,Diner
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Seafood Restaurant
19,Downtown Toronto,0,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Scenic Lookout,Restaurant,Brewery,Fried Chicken Joint,Pizza Place
17,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop,Gym / Fitness Center
11,Downtown Toronto,0,Coffee Shop,Bakery,Restaurant,Pizza Place,Pub,Café,Italian Restaurant,Convenience Store,Gastropub,Pharmacy


### Cluster 2

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Tennis Court,Summer Camp,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]